In [1]:
# !pip install uproot

In [2]:
# !pip install --target=$nb_path -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
# !pip install --target=$nb_path -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
# !pip install --target=$nb_path -q torch-geometric

In [3]:
# !pip install --target=$nb_path -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html

In [4]:
# !pip install torch-cluster

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime
import tqdm

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle
import torch_cluster
from torch_geometric.nn import GravNetConv

In [6]:
# getting the data

with open('/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/list_of_graphs_from_filtered_data_k10.pickle', 'rb') as f:
    list_of_graphs = pickle.load(f)

In [7]:
frac_test = 0.3
index_test_start = int(len(list_of_graphs) * (1 - frac_test))
list_of_graphs_training = list_of_graphs[:index_test_start]
list_of_graphs_test = list_of_graphs[index_test_start:]

In [8]:
k = 10
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.lin_00 = nn.Linear(in_features=13, out_features=64)
        self.ln_00 = nn.LayerNorm(64)
        self.lin_01 = nn.Linear(in_features=64, out_features=128)
        self.ln_01 = nn.LayerNorm(128)
        self.lin_02 = nn.Linear(in_features=128, out_features=13)
        self.ln_02 = nn.LayerNorm(13)

        self.conv1 = pyg_nn.GCNConv(in_channels=13, out_channels=13)
        self.conv2 = pyg_nn.GCNConv(in_channels=13, out_channels=13)

        self.gravnet1 = GravNetConv(in_channels=13, out_channels=13, space_dimensions=8, propagate_dimensions=7, k=k)
        self.gravnet2 = GravNetConv(in_channels=13, out_channels=13, space_dimensions=8, propagate_dimensions=7, k=k)

        self.lin1 = nn.Linear(in_features=13, out_features=256)
        self.ln1 = nn.LayerNorm(256)

        self.lin2 = nn.Linear(in_features=256, out_features=256)
        self.ln2 = nn.LayerNorm(256)

        self.lin3 = nn.Linear(in_features=256, out_features=1)

        self.relu = F.relu
        self.sigmoid = torch.sigmoid
        
    def forward(self, data):

        x = self.lin_00(data.x)
        x = self.relu(x)
        x = self.ln_00(x)
        x = self.lin_01(x)
        x = self.relu(x)
        x = self.ln_01(x)
        x = self.lin_02(x)
        x = self.relu(x)
        x = self.ln_02(x)
        
        x = self.gravnet1(data.x)
#         x = self.conv1(x=x, edge_index=data.edge_index)
        x = self.relu(x)
        
        x = self.gravnet2(x)
        # x = self.conv2(x=x, edge_index=data.edge_index)
        x = self.relu(x)

        x = self.lin1(x)
        x = self.relu(x)
        x = self.ln1(x)

        x = self.lin2(x)
        x = self.relu(x)
        x = self.ln2(x)

        x = self.lin3(x)

        out = self.sigmoid(x)

        return out

In [9]:
torch.cuda.is_available()

True

In [10]:
learning_rate = 0.01
num_epochs = 300

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel()
model = model.to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
accs = []
losses = []

for epoch in range(num_epochs):
    correct = 0
    correct_test = 0
    total = 0
    total_test = 0
    total_loss = 0
    total_loss_test = 0
    

    for graph in tqdm.tqdm(list_of_graphs_training):
        model.train()

        optimizer.zero_grad()
        graph.to(device)
        preds = model(graph)
        labels = graph.y
        loss = criterion(preds, labels)
        loss.backward()

        optimizer.step()

        total_loss += loss.item()
        correct += sum(abs((torch.round(preds) - labels)))
        total += graph.num_nodes
        
    for graph in list_of_graphs_test:
        model.eval()
        graph.to(device)
        preds = model(graph)
        labels = graph.y
        loss = criterion(preds, labels)

        total_loss_test += loss.item()
        correct_test += sum(abs((torch.round(preds) - labels)))
        total_test += graph.num_nodes
        
    
    accuracy = 1 - correct.item() / total
    total_loss /= len(list_of_graphs_training)
    accuracy_test = 1 - correct_test.item() / total_test
    total_loss_test /= len(list_of_graphs_test)
    accs.append(accuracy)
    losses.append(total_loss)
    print('Epoch: %d | Loss: %.8f | Train Accuracy: %.8f | Loss Test: %.8f | Test Accuracy: %.8f' \
          %(epoch, total_loss, accuracy, total_loss_test, accuracy_test))

100%|██████████████████████████████████████████████████████████████████████████████| 4312/4312 [00:42<00:00, 100.60it/s]


Epoch: 0 | Loss: 0.24197411 | Train Accuracy: 0.94560717 | Loss Test: 0.22222280 | Test Accuracy: 0.94678227


100%|██████████████████████████████████████████████████████████████████████████████| 4312/4312 [00:42<00:00, 101.29it/s]


Epoch: 1 | Loss: 0.19257607 | Train Accuracy: 0.94752239 | Loss Test: 0.17055855 | Test Accuracy: 0.95069104


100%|██████████████████████████████████████████████████████████████████████████████| 4312/4312 [00:42<00:00, 101.05it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 4312/4312 [00:41<00:00, 104.20it/s]


Epoch: 34 | Loss: 0.15511626 | Train Accuracy: 0.95917309 | Loss Test: 0.15049521 | Test Accuracy: 0.96000879


100%|██████████████████████████████████████████████████████████████████████████████| 4312/4312 [00:41<00:00, 104.10it/s]


Epoch: 35 | Loss: 0.15465656 | Train Accuracy: 0.95935951 | Loss Test: 0.14994428 | Test Accuracy: 0.96029869


 36%|████████████████████████████                                                   | 1535/4312 [00:15<00:28, 98.29it/s]

In [ ]:
# model_name = "GCNConv"
# model_name_cat = "GCNConv"
model_name = f"GravNet_k{K}"
model_name_cat = f"GravNet_k{K}"

In [ ]:
preds = []
labels = []

for graph in list_of_graphs_test:
  graph.to(device)
  pred = model(graph)
  label = graph.y
  preds.append(pred)
  labels.append(label)

In [ ]:
preds = torch.cat(preds).to("cpu").detach().numpy()
labels = torch.cat(labels).to("cpu").detach().numpy()

In [ ]:
plt.title(f"{model_name}")
plt.plot(losses, label="training loss")
plt.plot(accs, label="training accuracy")
plt.legend()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

fpr_keras, tpr_keras, thresholds_keras = roc_curve(labels, preds)
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='(AUC = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title(f'ROC curve ({model_name})')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, f1_score, auc

precision, recall, thresholds = precision_recall_curve(labels, preds)
auc = auc(recall, precision)

no_skill = len(labels[labels==1]) / len(labels)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(recall, precision, marker='.', label=f'{model_name}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.text(0.8, 0.8, '(AUC = {:.3f})'.format(auc))
plt.title(f"{model_name} precision-recall curve")
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(), f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/{model_name_cat}_150epochs_first_save.pickle')

In [ ]:
with open(f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/{model_name_cat}_150epochs_first_save_accs.pickle', 'wb') as handle:
    pickle.dump(accs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/{model_name_cat}_150epochs_first_save_losses.pickle', 'wb') as handle:
    pickle.dump(losses, handle, protocol=pickle.HIGHEST_PROTOCOL)